## Data Unit Standardization

This notebook performs unit standardization and category harmonization
across different futures contracts.

Specifically:
- All futures prices are converted to a common currency (USD).
- Futures of the same energy type are standardized to consistent
  capacity or weight units.
- Petroleum-related products are further categorized into
  crude oil and refined oil products.

These transformations ensure cross-market comparability and
reduce distortions caused by heterogeneous contract specifications.

In [14]:
import pandas as pd
import os

In [15]:
# Loading
# ----------------------------

df = pd.read_csv(r'./data/Raw_EUA.csv')
future = df.copy()

print(df.head)

<bound method NDFrame.head of              Time        Open        High         Low        Last  Change  \
0      2005-04-25   17.150000   17.350000   17.150000   17.450000     NaN   
1      2005-04-26   17.450000   17.450000   17.100000   17.150000   -0.30   
2      2005-04-27   17.100000   17.100000   16.550000   16.350000   -0.80   
3      2005-04-28   16.300000   16.500000   15.750000   15.950000   -0.40   
4      2005-04-29   16.500000   16.500000   16.500000   16.400000    0.45   
...           ...         ...         ...         ...         ...     ...   
42389  2025-11-07  115.678113  115.863106  113.990057  114.533472   -1.34   
42390  2025-11-07  130.963117  130.963117  130.963117  130.963117   -1.34   
42391  2025-11-07  132.136614  132.136614  132.136614  132.136614   -1.75   
42392  2025-11-07   89.120130   89.120130   89.120130   89.120130   -1.34   
42393  2025-11-07   59.620000   60.460000   59.330000   59.750000    0.32   

         Volume  Open Int  Change %  Contract

In [16]:
# Re-label petroleum categories
# ----------------------------

mask_crude = future["Source"].isin(["CBF", "WIZ"])
mask_refined = future["Source"].isin(["LFZ", "LGZ", "LOZ"])

future.loc[mask_crude, 'Commodity'] = "crude oil"
future.loc[mask_refined, 'Commodity'] = "refined oil"

print(future[mask_crude | mask_refined])

             Time      Open      High      Low      Last  Change     Volume  \
3980   2016-11-23   58.8100   58.8100   58.810   58.8100 -0.3900   139733.0   
3981   2016-11-24   58.7700   58.7700   58.770   58.7700 -0.0400    18085.0   
3983   2016-11-25   57.4700   57.4700   57.470   57.4700 -1.3000    87582.0   
3988   2016-11-28   58.0000   58.0000   58.000   58.0000  0.5300   200429.0   
3989   2016-11-29   56.6600   56.6600   56.660   56.6600 -1.3400   215301.0   
...           ...       ...       ...      ...       ...     ...        ...   
42363  2025-11-07   63.6300   64.3900   63.230   63.6300  0.2500  1061480.0   
42364  2025-11-07    1.9685    1.9878    1.935    1.9403 -0.0253   120312.0   
42365  2025-11-07    1.9685    1.9878    1.935    1.9403 -0.0253   120312.0   
42366  2025-11-07  729.2500  747.5000  717.500  725.2500  3.2500   572008.0   
42393  2025-11-07   59.6200   60.4600   59.330   59.7500  0.3200   233630.0   

       Open Int  Change %  Contract_size Source    

## Currency Standardization

All futures prices are converted to a common currency (USD) to ensure
cross-market comparability.

Daily FX rates are merged by date, and only contracts denominated in
non-USD currencies are converted. After conversion, unit labels are
updated accordingly and FX-rate columns are removed.

This step prevents exchange-rate differences from dominating the
subsequent multivariate analysis.



In [17]:
def convert_to_usd(df, rate_df, cur="EUR", date_col="Time", currency_col="Contract_size", price_cols=("Open", "High", "Low", "Last")):
    
    df = df.copy()
    rate = rate_df.copy()

    # turn time from string to timestamp
    df[date_col] = pd.to_datetime(df[date_col], utc=True, errors="coerce").dt.date
    rate['Date'] = pd.to_datetime(rate['Date'], utc=True, errors="coerce").dt.date
    
    # merge original and rate
    rate = rate.rename(columns={"Date": date_col})
    df = df.merge(rate, on=date_col, how="left")

    # rows to convert
    mask_cur = df[currency_col].astype(str).str.startswith(cur)

    # missing FX check
    col_name = cur + "USD=X"
    if df.loc[mask_cur, col_name].isna().any():
        missing_dates = df.loc[mask_cur & df[col_name].isna(), date_col].unique()
        print(f"[WARN] Missing {cur}/USD rates for dates (show up to 10): {missing_dates[:10]}")

    # convert price columns
    col_use = [c for c in price_cols if c in df.columns]
    df.loc[mask_cur, col_use] = (
        df.loc[mask_cur, col_use]
        .mul(df.loc[mask_cur, col_name], axis=0)
    )
    
    # update unit label
    df.loc[mask_cur, currency_col] = df.loc[mask_cur, currency_col].str.replace(
        cur, "USD", n=1
    )

    # drop the rate col
    df = df.drop(columns=[col_name])

    return df

In [19]:
# FX files
# ----------------------------

eur = pd.read_csv(r'.\data\exchange\EX_EURUSD.csv')
gbp = pd.read_csv(r'.\data\exchange\EX_GBPUSD.csv')

future = convert_to_usd(future, eur, cur="EUR")
future = convert_to_usd(future, gbp, cur="GBP")

mask_eur = df["Contract_size"].astype(str).str.contains("EUR")
mask_gbp = df["Contract_size"].astype(str).str.contains("GBP")

print(df[mask_eur])
print(future[mask_eur])
print(future[mask_gbp])


             Time    Open    High     Low    Last  Change    Volume  \
6567   2019-06-04  17.760  17.760  17.760  17.760   0.445   89599.0   
6583   2019-06-05  17.638  17.638  17.638  17.638  -0.122   68820.0   
6584   2019-06-06  17.603  17.603  17.603  17.603  -0.035   61677.0   
6596   2019-06-07  17.881  17.881  17.881  17.881   0.278   41465.0   
6606   2019-06-10  18.169  18.169  18.169  18.169   0.288   69319.0   
...           ...     ...     ...     ...     ...     ...       ...   
42361  2025-11-06  32.750  32.850  32.648  32.648  -0.271     250.0   
42373  2025-11-07  30.265  30.265  30.265  30.265  -0.375       0.0   
42375  2025-11-07  32.301  32.301  32.301  32.301  -0.347     180.0   
42376  2025-11-07  31.700  31.700  31.095  31.201  -0.347  249708.0   
42377  2025-11-07  33.936  33.936  33.936  33.936  -0.347       0.0   

        Open Int  Change % Contract_size Source   Commodity  
6567   1091329.0       NaN     EUR / MWh    TGZ  nature gas  
6583   1092624.0 -0.006

In [20]:
# Gas unit standardization to USD/MWh
# ----------------------------

price_cols = ["Open", "High", "Low", "Last"]
col_use = [c for c in price_cols if c in future.columns]

# robust commodity match
mask_gas = future["Commodity"] == "nature gas"
mask_mmbtu = mask_gas & (future["Contract_size"] == "USD / MMBtu")
mask_therm = mask_gas & (future["Contract_size"] == "USD / Therm")

gas_unit = df.loc[mask_gas, "Contract_size"].unique()

# USD/MMBtu -> USD/MWh (1 MWh = 3.412141633 MMBtu)
future.loc[mask_mmbtu, "Contract_size"] = "USD / MWh"
future.loc[mask_mmbtu, "Last"] *= 3.412141633

# USD/Therm -> USD/MWh (1 Therm = 0.029307107 MWh => /0.029307107 = 34.121416...)
future.loc[mask_therm, "Contract_size"] = "USD / MWh"
future.loc[mask_therm, "Last"] *= 34.121416

print("Gas units after conversion:", future.loc[mask_gas, "Contract_size"].unique())

Gas units after conversion: ['USD / MWh']


In [10]:
# Coal naming cleanup
# ----------------------------

future.loc[future["Commodity"].astype(str).str.lower() == "coal", "Commodity"] = "coal"

In [ ]:
# Save
# ----------------------------

os.makedirs(".\data", exist_ok=True)
future.to_csv(".\data\Energy Future.csv", index=False)